In [53]:
import numpy as np

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import pandas as pd

from scipy.spatial.distance import euclidean
from numpy import array, zeros, argmin, inf, equal, ndim
from scipy.spatial.distance import cdist

from sklearn import svm
from sklearn.svm import LinearSVC

In [2]:
d1 = glob.glob('C:/Users/iNitesh/Dropbox/Datasets/digit_recognition_speech/data/isolated/36/1/*')
d2 = glob.glob('C:/Users/iNitesh/Dropbox/Datasets/digit_recognition_speech/data/isolated/36/2/*')
d3 = glob.glob('C:/Users/iNitesh/Dropbox/Datasets/digit_recognition_speech/data/isolated/36/3/*')

In [3]:
def load_mfcc(list_paths):
    X = []
    for i in list_paths:
        X.append(np.array(pd.read_csv(i, sep = " ", skiprows = [0], header = None))[:,1:])

    return np.array(X)

def combine(X):
    temp_X = []

    for i in X:
        for j in i:
            temp_X.append(j)

    return np.array(temp_X)

In [4]:
# Load and structure data
X1 = load_mfcc(d1)
X2 = load_mfcc(d2)
X3 = load_mfcc(d3)

# Train Test data partition
# X1_train,X1_test = train_test_split(X1, test_size=0.3, random_state=42)
# X2_train,X2_test = train_test_split(X2, test_size=0.3, random_state=42)
# X3_train,X3_test = train_test_split(X3, test_size=0.3, random_state=42)

In [5]:
def _traceback(D):
    i, j = array(D.shape) - 2
    p, q = [i], [j]
    while ((i > 0) or (j > 0)):
        tb = argmin((D[i, j], D[i, j+1], D[i+1, j]))
        if (tb == 0):
            i -= 1
            j -= 1
        elif (tb == 1):
            i -= 1
        else: # (tb == 2):
            j -= 1
        p.insert(0, i)
        q.insert(0, j)
    return array(p), array(q)

In [6]:
def dtw(x, y, dist):
    """
    Computes Dynamic Time Warping (DTW) of two sequences.

    :param array x: N1*M array
    :param array y: N2*M array
    :param func dist: distance used as cost measure

    Returns the minimum distance, the cost matrix, the accumulated cost matrix, and the wrap path.
    """
    assert len(x)
    assert len(y)
    r, c = len(x), len(y)
    D0 = zeros((r + 1, c + 1))
    D0[0, 1:] = inf
    D0[1:, 0] = inf
    D1 = D0[1:, 1:] # view
    for i in range(r):
        for j in range(c):
            D1[i, j] = dist(x[i], y[j])
    C = D1.copy()
    for i in range(r):
        for j in range(c):
            D1[i, j] += min(D0[i, j], D0[i, j+1], D0[i+1, j])
    if len(x)==1:
        path = zeros(len(y)), range(len(y))
    elif len(y) == 1:
        path = range(len(x)), zeros(len(x))
    else:
        path = _traceback(D0)
    return (D1[-1, -1] / sum(D1.shape), C, D1, path)

In [7]:
def fastdtw(x, y, dist):
    """
    Computes Dynamic Time Warping (DTW) of two sequences in a faster way.
    Instead of iterating through each element and calculating each distance,
    this uses the cdist function from scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html)

    :param array x: N1*M array
    :param array y: N2*M array
    :param string or func dist: distance parameter for cdist. When string is given, cdist uses optimized functions for the distance metrics.
    If a string is passed, the distance function can be 'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'wminkowski', 'yule'.
    Returns the minimum distance, the cost matrix, the accumulated cost matrix, and the wrap path.
    """
    assert len(x)
    assert len(y)
    if ndim(x)==1:
        x = x.reshape(-1,1)
    if ndim(y)==1:
        y = y.reshape(-1,1)
    r, c = len(x), len(y)
    D0 = zeros((r + 1, c + 1))
    D0[0, 1:] = inf
    D0[1:, 0] = inf
    D1 = D0[1:, 1:]
    D0[1:,1:] = cdist(x,y,dist)
    C = D1.copy()
    for i in range(r):
        for j in range(c):
            D1[i, j] += min(D0[i, j], D0[i, j+1], D0[i+1, j])
    if len(x)==1:
        path = zeros(len(y)), range(len(y))
    elif len(y) == 1:
        path = range(len(x)), zeros(len(x))
    else:
        path = _traceback(D0)
    return (D1[-1, -1] / sum(D1.shape), C, D1, path)

In [8]:
from sklearn.metrics.pairwise import euclidean_distances
from nltk.metrics.distance import edit_distance
from sklearn.metrics.pairwise import manhattan_distances

#dist_fun = euclidean_distances

In [9]:
max = 0
n_frames_1 = []
n_frames_2 = []
n_frames_3 = []
for i in range(0,57):
    n_frames_1.append(X1[i].shape[0])
    n_frames_2.append(X2[i].shape[0])
    n_frames_3.append(X3[i].shape[0])
print(np.max(np.array(n_frames_1)))
print(np.max(np.array(n_frames_2)))
print(np.max(np.array(n_frames_3)))

print(n_frames_1)
print()
print(n_frames_2)
print()
print(n_frames_3)

size = []
for each in n_frames_1:
    if each in n_frames_2 and each in n_frames_3:
        size.append(each)
print(np.max(size))
print(n_frames_1.index(124))
print(n_frames_2.index(124))
print(n_frames_3.index(124))

194
131
183
[75, 84, 79, 84, 101, 97, 116, 94, 101, 94, 103, 194, 108, 88, 85, 89, 70, 97, 97, 92, 97, 96, 97, 121, 85, 94, 108, 75, 97, 92, 124, 112, 78, 81, 107, 84, 110, 78, 85, 102, 103, 80, 93, 88, 94, 85, 88, 84, 80, 94, 121, 98, 103, 80, 90, 89, 85]

[70, 83, 80, 83, 106, 103, 120, 98, 87, 107, 101, 88, 79, 84, 83, 72, 131, 87, 75, 88, 117, 92, 98, 98, 78, 124, 85, 84, 76, 93, 106, 93, 99, 85, 103, 87, 113, 78, 97, 85, 106, 101, 84, 84, 67, 83, 79, 84, 80, 92, 97, 79, 78, 80, 74, 117, 78]

[71, 88, 83, 107, 103, 88, 106, 108, 89, 181, 130, 93, 83, 81, 83, 83, 94, 93, 101, 88, 94, 112, 116, 183, 80, 83, 112, 96, 101, 94, 124, 106, 81, 87, 104, 89, 92, 85, 106, 145, 129, 96, 133, 79, 143, 99, 79, 88, 78, 98, 119, 92, 85, 83, 74, 135, 81]
124
30
25
30


In [26]:
ref_1 = X1[30]
ref_2 = X2[25]
ref_3 = X3[30]

X = []
Y = []
for i in range(0, 57):
    test = X1[i]
    dist_fun = lambda template, test: np.linalg.norm(template - test, ord=1)
    dist, cost, acc, path = fastdtw(ref_1, test, dist_fun)
    tmp = path[1]
    
    fixed_mfcc = np.zeros((124, 38))
    for j in range(0, 124):
        fixed_mfcc[j] = test[tmp[j]]
    X.append(fixed_mfcc)
    Y.append(1)

for i in range(0, 57):
    test = X2[i]
    dist_fun = lambda template, test: np.linalg.norm(template - test, ord=1)
    dist, cost, acc, path = fastdtw(ref_2, test, dist_fun)
    tmp = path[1]
    
    fixed_mfcc = np.zeros((124, 38))
    for j in range(0, 124):
        fixed_mfcc[j] = test[tmp[j]]
    X.append(fixed_mfcc)
    Y.append(2)
        
for i in range(0, 57):
    test = X3[i]
    dist_fun = lambda template, test: np.linalg.norm(template - test, ord=1)
    dist, cost, acc, path = fastdtw(ref_3, test, dist_fun)
    tmp = path[1]
    
    fixed_mfcc = np.zeros((124, 38))
    for j in range(0, 124):
        fixed_mfcc[j] = test[tmp[j]]        
    X.append(fixed_mfcc)
    Y.append(3)
    #np.savetxt('C:/Users/iNitesh/Desktop/SVM/fixed_length_mfcc/3/%d.txt' % i, fixed_mfcc)
    

In [31]:
X = np.array(X).reshape(-1, 124*38)
Y = np.array(Y)

(171,)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [54]:
clf = LinearSVC(random_state=0)
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

In [59]:
clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': 0,
 'tol': 0.0001,
 'verbose': 0}

In [78]:
clf =svm.SVC(kernel= 'sigmoid', probability=True)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [79]:
y_pred == y_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True], dtype=bool)

In [80]:
clf.score(X_test, y_test)

0.96491228070175439

In [81]:
clf.n_support_
clf.support_

array([  6,  19,  36,  51,  55,  56,  59,  61,  72,  83,  85,  88,  97,
       108, 111,   3,  20,  27,  49,  63,  70,  82, 102, 105, 112,  10,
        11,  14,  26,  33,  34,  44,  46,  48,  64,  69,  74,  75,  78,
        79,  81,  86, 106])

In [85]:
clf.predict_proba(X_test)

array([[  2.92275273e-02,   7.48734716e-01,   2.22037757e-01],
       [  8.37549785e-01,   3.52312305e-02,   1.27218985e-01],
       [  9.12938253e-01,   2.40728713e-02,   6.29888756e-02],
       [  5.39993813e-02,   6.18060141e-03,   9.39820017e-01],
       [  3.40577257e-02,   7.34175324e-03,   9.58600521e-01],
       [  9.65230181e-03,   9.25638329e-01,   6.47093688e-02],
       [  2.41520013e-02,   1.75890609e-01,   7.99957390e-01],
       [  8.55159234e-03,   9.79532194e-01,   1.19162135e-02],
       [  8.44186846e-03,   9.51409354e-01,   4.01487775e-02],
       [  4.50311888e-03,   9.93791926e-01,   1.70495531e-03],
       [  1.10779745e-01,   1.53490903e-02,   8.73871165e-01],
       [  3.08278722e-03,   9.96396206e-01,   5.21006504e-04],
       [  9.82222305e-01,   2.98206019e-03,   1.47956346e-02],
       [  9.77124869e-01,   4.23038376e-03,   1.86447475e-02],
       [  9.69625475e-01,   6.57518975e-03,   2.37993353e-02],
       [  9.51195684e-01,   1.98964333e-02,   2.8907882